### import libraries

In [4]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import datetime

### load user dataset

In [172]:
df_rachel = pd.read_csv(r"C:\Users\ralexander\Downloads\rachel_notion_recommender_engine.csv")

In [173]:
df_rachel.head()
## pd.unique(df_rachel['Type'])

,Name,Type,Genre
0,10 Things I Hate About You,Movie,"Comedy, Romance, Teen"
1,17 Again,Movie,"Comedy, Fantasy, Feel-Good, Romance"
2,A Good Girl’s Guide to Murder,Book,"Mystery, Teen"
3,A Thousand Splendid Suns,Book,"Drama, Friendship, Historical, REAL"
4,Abbott Elementary,TV Show,"Comedy, Sit-com"


#### subset dataset

In [174]:
# Book df
df_books = df_rachel[df_rachel['Type'].str.contains('Book')]
# Movie df
df_movie = df_rachel[df_rachel['Type'].str.contains('Movie')]
# TV Show df
df_tvshow = df_rachel[df_rachel['Type'].str.contains('TV Show')]

df_books.head(50)

,Name,Type,Genre
2,A Good Girl’s Guide to Murder,Book,"Mystery, Teen"
3,A Thousand Splendid Suns,Book,"Drama, Friendship, Historical, REAL"
6,And Then There Were None,Book,"Dark, Mystery"
9,Aristotle and Dante Discover the Secrets of th...,Book,"Family, Friendship, REAL"
19,Complications,Book,REAL
22,Educated,Book,"Family, REAL"
23,Eleanor & Park,Book,"Drama, Romance, Teen"
24,Ender’s Game,Book,"Dark, Drama, Sci-Fi, Society"
25,Eragon,Book,"Action and Adventure, Fantasy, Kids"
30,Half of a Yellow Sun,Book,"Historical, Romance, Society"


### NYT Bestsellers Book Reviews

In [175]:
## subset top 10 of each category for text analytics
### text is applicable to books, movie and tb scripts, music lyrics, song titles

## text sentiment analysis
## text matching
## topic frequency

## start: topic frequency among book reviews

# what are the different features of text data that can apply to books and movies?

In [176]:
# NYT api practice

title = "Iron%20Flame"
api = f"https://api.nytimes.com/svc/books/v3/reviews.json?title={title}&api-key=B6sstxBJ39IKPTsYZA72UGVPhBGymY7G"

response = requests.get(api)
data = response.json()

## print(data)

review = data#["results"]#[0]["summary"]

#print(review)

print(json.dumps(review, indent=4, sort_keys=True)) # not every NYT Bestseller has book reviews! (weird)


{
    "copyright": "Copyright (c) 2024 The New York Times Company.  All Rights Reserved.",
    "num_results": 0,
    "results": [],
    "status": "OK"
}


In [177]:
genre = "mass-market-paperback" 
api = f'https://api.nytimes.com/svc/books/v3/lists.json?list={genre}&api-key=B6sstxBJ39IKPTsYZA72UGVPhBGymY7G'

response = requests.get(api)
data = response.json()

print(json.dumps(data["results"][0]["book_details"][0]["title"], indent=4, sort_keys=True))
# print(json.dumps(data2["results"][0]["book_details"][0]["description"], indent=4, sort_keys=True))

# FINDING: none of these have book reviews. Pivot to analyzing book summaries?

"SWEET TOMORROWS"


In [178]:
# Build df 

# Populate list
title_lst = []
author_lst = []
summary_lst = []
isbn10_lst = []
isbn13_lst = []
# publish_date_lst = []
# genre_lst = []
# amazon_url_lst = []

for i in range(0, len(data["results"])):
    title_lst.append(json.dumps(data["results"][i]["book_details"][0]["title"], indent=4, sort_keys=True)) # title
    author_lst.append(json.dumps(data["results"][i]["book_details"][0]["author"], indent=4, sort_keys=True)) # author
    summary_lst.append(json.dumps(data["results"][i]["book_details"][0]["description"], indent=4, sort_keys=True)) # summary
    isbn10_lst.append(json.dumps(data["results"][i]["book_details"][0]["primary_isbn10"], indent=4, sort_keys=True)) # isbn10
    isbn13_lst.append(json.dumps(data["results"][i]["book_details"][0]["primary_isbn13"], indent=4, sort_keys=True)) # isbn13
    # publish_date_lst.append(json.dumps(data["results"][i]["published_date"], indent=4, sort_keys=True)) # publish date
    # genre_lst.append(json.dumps(data["results"][i]["list_name"], indent=4, sort_keys=True)) # publish date
    # amazon_url_lst.append(json.dumps(data["results"][i]["amazon_product_url"], indent=4, sort_keys=True)) # amazon product url

# Initialize df
nytimes1204_dict = {
    'title': title_lst
    , 'author': author_lst
    , 'summary': summary_lst
    , 'isbn10': isbn10_lst
    , 'isbn13': isbn13_lst
    # , 'publish_date': publish_date_lst
    # , 'genre': genre_lst
    # , 'amazon_url': amazon_url_lst
}

nytimes1204_df = pd.DataFrame(nytimes1204_dict)

In [180]:
nytimes1204_df['title'] = nytimes1204_df['title'].str.strip('"')
nytimes1204_df['author'] = nytimes1204_df['author'].str.strip('"')
nytimes1204_df['summary'] = nytimes1204_df['summary'].str.strip('"')
nytimes1204_df['isbn10'] = nytimes1204_df['isbn10'].str.strip('"')
nytimes1204_df['isbn13'] = nytimes1204_df['isbn13'].str.strip('"')
nytimes1204_df.head()

,title,author,summary,isbn10,isbn13
0,SWEET TOMORROWS,Debbie Macomber,The journeys of the characters at the Rose Har...,0553391860,9780553391862
1,THE MURDER HOUSE,James Patterson and David Ellis,When bodies are found at a Hamptons estate whe...,1455589918,9781455589913
2,APPRENTICE IN DEATH,J D Robb,Lt. Eve Dallas of the N.Y.P.D. is on the case ...,1101987995,9781101987995
3,A DOG'S PURPOSE,W Bruce Cameron,"From stray mutt to golden-haired puppy, a dog ...",0765388103,9780765388100
4,MULBERRY MOON,Catherine Anderson,Ben Sterling returns home to Mystic Creek to s...,0451488024,9780451488022


In [ ]:
# clean json response into PANDAs dataframe for current subset - CHECK
# write function CHECK
# use function iteratively to extract data over a period of time - nested loop over period of genres
    # best sellters list is updated online every Wednesday at 7pm EST
# Exploratory data analysis 
    # underlying modeling techniques. What kind of cool data science analysis can we apply

# check google books api


In [2]:
# Define get book details function

def NYT_book_details(date, NYT_category):
    """
    Populate a pandas DataFrame with details on the NYT bestsellers of a given week for a particular genre

    Parameters:
    date: str format YYYY-MM-DD (Sunday). Reflects sales for week ending two weeks prior (Saturday)
    NYT_category: str of NYT accepted category. Formatted with spaces replaced by hyphens.

    Returns:
    a populated pandas DataFrame
    """
    
    # Initialize list containers for book details
    title_lst = []
    author_lst = []
    summary_lst = []
    isbn10_lst = []
    isbn13_lst = []

    # Authorize NYT Bestsellers API
    api = f'https://api.nytimes.com/svc/books/v3/lists/{date}/{NYT_category}.json?&api-key=B6sstxBJ39IKPTsYZA72UGVPhBGymY7G'
    response = requests.get(api)
    data = response.json()

    # Check if api call retrieves results
    # if data['results'] != []:

    # Populate lists of book details
    for i in range(0, len(data["results"])):
        title_lst.append(json.dumps(data["results"]["books"][i]["title"], indent=4, sort_keys=True)) # title
        author_lst.append(json.dumps(data["results"]["books"][i]["author"], indent=4, sort_keys=True)) # author
        summary_lst.append(json.dumps(data["results"]["books"][i]["description"], indent=4, sort_keys=True)) # summary
        isbn10_lst.append(json.dumps(data["results"]["books"][i]["primary_isbn10"], indent=4, sort_keys=True)) # isbn10
        isbn13_lst.append(json.dumps(data["results"]["books"][i]["primary_isbn13"], indent=4, sort_keys=True)) # isbn13

    # Create DataFrame
    book_details_dict = {
        'title': title_lst
        , 'author': author_lst
        , 'summary': summary_lst
        , 'isbn10': isbn10_lst
        , 'isbn13': isbn13_lst
    }

    book_details_df = pd.DataFrame(book_details_dict)

    try: 
        book_details_df['title'] = book_details_df['title'].str.strip('"')
        book_details_df['author'] = book_details_df['author'].str.strip('"')
        book_details_df['summary'] = book_details_df['summary'].str.strip('"')
        book_details_df['isbn10'] = book_details_df['isbn10'].str.strip('"')
        book_details_df['isbn13'] = book_details_df['isbn13'].str.strip('"')

        sales_ending_week = datetime.datetime.strptime(date, "%Y-%m-%d") - timedelta(days=15) # calcuate date of week when sales end
        
        book_details_df['sales_ending_week'] = sales_ending_week.strftime("%Y-%m-%d")
        book_details_df['NYT_category'] = NYT_category
    except AttributeError:
        pass

    return book_details_df

In [182]:
NYT_book_details("2024-12-01", "hardcover-nonfiction")

,title,author,summary,isbn10,isbn13,sales_ending_week,NYT_category
0,MELANIA,Melania Trump,The former and future first lady describes her...,1510782699,9781510782693,2024-11-16,hardcover-nonfiction
1,FRAMED,John Grisham and Jim McCloskey,Our criminal justice system viewed through the...,0385550448,9780385550444,2024-11-16,hardcover-nonfiction
2,THE ANXIOUS GENERATION,Jonathan Haidt,A co-author of \u201cThe Coddling of the Ameri...,0593655036,9780593655030,2024-11-16,hardcover-nonfiction
3,THE WAR ON WARRIORS,Pete Hegseth,"The former \""Fox & Friends Weekend\"" host shar...",0063389428,9780063389427,2024-11-16,hardcover-nonfiction
4,CONFRONTING THE PRESIDENTS,Bill O'Reilly and Martin Dugard,The conservative commentator evaluates the leg...,125034641X,9781250346414,2024-11-16,hardcover-nonfiction
5,REVENGE OF THE TIPPING POINT,Malcolm Gladwell,"Through a series of stories, Gladwell explicat...",0316575801,9780316575805,2024-11-16,hardcover-nonfiction
6,THE MESSAGE,Ta-Nehisi Coates,The author of \u201cBetween the World and Me\u...,0593230388,9780593230381,2024-11-16,hardcover-nonfiction
7,FROM HERE TO THE GREAT UNKNOWN,Lisa Marie Presley and Riley Keough,"Presley\u2019s memoir, completed by her daught...",0593733878,9780593733875,2024-11-16,hardcover-nonfiction
8,BE READY WHEN THE LUCK HAPPENS,Ina Garten,A memoir by the cookbook author and Food Netwo...,0593799895,9780593799895,2024-11-16,hardcover-nonfiction
9,WAR,Bob Woodward,The Pulitzer Prize-winning journalist looks at...,166805227X,9781668052273,2024-11-16,hardcover-nonfiction


In [183]:
sales_ending_week = datetime.datetime.strptime("2024-12-01", "%Y-%m-%d") - timedelta(days=15)
sales_ending_week.strftime("%Y-%m-%d")

'2024-11-16'

In [184]:
# Get full list of NYT categories
api = f'https://api.nytimes.com/svc/books/v3/lists/names.json?&api-key=B6sstxBJ39IKPTsYZA72UGVPhBGymY7G'
response = requests.get(api)
data = response.json()

for i in range(0, len(data["results"])):
    print(data["results"][i]["list_name_encoded"])
    #print(type(x))


combined-print-and-e-book-fiction
combined-print-and-e-book-nonfiction
hardcover-fiction
hardcover-nonfiction
trade-fiction-paperback
mass-market-paperback
paperback-nonfiction
e-book-fiction
e-book-nonfiction
hardcover-advice
paperback-advice
advice-how-to-and-miscellaneous
hardcover-graphic-books
paperback-graphic-books
manga
combined-print-fiction
combined-print-nonfiction
chapter-books
childrens-middle-grade
childrens-middle-grade-e-book
childrens-middle-grade-hardcover
childrens-middle-grade-paperback
paperback-books
picture-books
series-books
young-adult
young-adult-e-book
young-adult-hardcover
young-adult-paperback
animals
audio-fiction
audio-nonfiction
business-books
celebrities
crime-and-punishment
culture
education
espionage
expeditions-disasters-and-adventures
fashion-manners-and-customs
food-and-fitness
games-and-activities
graphic-books-and-manga
hardcover-business-books
health
humor
indigenous-americans
relationships
mass-market-monthly
middle-grade-paperback-monthly
pape

In [185]:
week_date_counter = '2024-12-01'
week_dttm_counter = datetime.datetime.strptime(week_date_counter, '%Y-%m-%d')

sunday_end_date = '2025-01-05'
sunday_end_dttm = datetime.datetime.strptime(sunday_end_date, '%Y-%m-%d')

In [7]:
NYT_bestsellers_24 = pd.DataFrame()

# Get list of NYT cateogries
api = f'https://api.nytimes.com/svc/books/v3/lists/names.json?&api-key=B6sstxBJ39IKPTsYZA72UGVPhBGymY7G'
response = requests.get(api)
data = response.json()

# Initialize timeframe dates
week_date_counter = '2024-12-01'
week_dttm_counter = datetime.datetime.strptime(week_date_counter, '%Y-%m-%d')

sunday_end_date = '2025-01-05'
sunday_end_dttm = datetime.datetime.strptime(sunday_end_date, '%Y-%m-%d')

# Run function
while week_dttm_counter <= sunday_end_dttm:
    week_date_counter = week_dttm_counter.strftime('%Y-%m-%d')
    print(week_date_counter)
    for i in range(0, len(data["results"])):
        print(data["results"][i]["list_name_encoded"])
        NYT_bestsellers_24 = pd.concat([NYT_bestsellers_24, NYT_book_details(week_date_counter, data["results"][i]["list_name_encoded"])], ignore_index=True)
        
    week_date_counter += timedelta(weeks=1)
    print("loop continues")

print("loop complete")

2024-12-01
combined-print-and-e-book-fiction
combined-print-and-e-book-nonfiction
hardcover-fiction
hardcover-nonfiction
trade-fiction-paperback


KeyError: 'results'

In [5]:
NYT_book_details("2024-12-01", "trade-fiction-paperback")

,title,author,summary,isbn10,isbn13,sales_ending_week,NYT_category
0,THE BOYFRIEND,Freida McFadden,A series of recent deaths causes Sydney Shaw t...,1728296226,9781728296227,2024-11-16,trade-fiction-paperback
1,A COURT OF THORNS AND ROSES,Sarah J. Maas,"After killing a wolf in the woods, Feyre is ta...",1635575567,9781635575569,2024-11-16,trade-fiction-paperback
2,THE HOUSEMAID,Freida McFadden,Troubles surface when a woman looking to make ...,1538742578,9781538742570,2024-11-16,trade-fiction-paperback
3,FOURTH WING,Rebecca Yarros,Violet Sorrengail is urged by the commanding g...,1649377371,9781649377371,2024-11-16,trade-fiction-paperback
4,HOW MY NEIGHBOR STOLE CHRISTMAS,Meghan Quinn,A fake relationship and a Christmas contest ma...,1464230153,9781464230158,2024-11-16,trade-fiction-paperback
5,THE FROZEN RIVER,Ariel Lawhon,"In Maine, 1789, a midwife seeks to uncover the...",0593312074,9780593312070,2024-11-16,trade-fiction-paperback
6,THE HANDMAID'S TALE,Margaret Atwood,"In the Republic of Gilead's dystopian future, ...",038549081X,9780385490818,2024-11-16,trade-fiction-paperback
7,HEXED,Emily McIntire,The sixth book in the Never After series. A fo...,1728275865,9781728275864,2024-11-16,trade-fiction-paperback
8,THE HOUSEMAID'S SECRET,Freida McFadden,The second book in the Housemaid series. The s...,0349132615,9780349132617,2024-11-16,trade-fiction-paperback
9,LIGHTS OUT,Navessa Allen,"As Aly and Josh live out their dark fantasies,...",1638932239,9781638932239,2024-11-16,trade-fiction-paperback


In [8]:
api = f'https://api.nytimes.com/svc/books/v3/lists/{'2024-12-01'}/{'trade-fiction-paperback'}.json?&api-key=B6sstxBJ39IKPTsYZA72UGVPhBGymY7G'
response = requests.get(api)
data = response.json()
#data["results"]['books']

In [10]:
NYT_book_details('2024-12-01', 'trade-fiction-paperback')

,title,author,summary,isbn10,isbn13,sales_ending_week,NYT_category
0,THE BOYFRIEND,Freida McFadden,A series of recent deaths causes Sydney Shaw t...,1728296226,9781728296227,2024-11-16,trade-fiction-paperback
1,A COURT OF THORNS AND ROSES,Sarah J. Maas,"After killing a wolf in the woods, Feyre is ta...",1635575567,9781635575569,2024-11-16,trade-fiction-paperback
2,THE HOUSEMAID,Freida McFadden,Troubles surface when a woman looking to make ...,1538742578,9781538742570,2024-11-16,trade-fiction-paperback
3,FOURTH WING,Rebecca Yarros,Violet Sorrengail is urged by the commanding g...,1649377371,9781649377371,2024-11-16,trade-fiction-paperback
4,HOW MY NEIGHBOR STOLE CHRISTMAS,Meghan Quinn,A fake relationship and a Christmas contest ma...,1464230153,9781464230158,2024-11-16,trade-fiction-paperback
5,THE FROZEN RIVER,Ariel Lawhon,"In Maine, 1789, a midwife seeks to uncover the...",0593312074,9780593312070,2024-11-16,trade-fiction-paperback
6,THE HANDMAID'S TALE,Margaret Atwood,"In the Republic of Gilead's dystopian future, ...",038549081X,9780385490818,2024-11-16,trade-fiction-paperback
7,HEXED,Emily McIntire,The sixth book in the Never After series. A fo...,1728275865,9781728275864,2024-11-16,trade-fiction-paperback
8,THE HOUSEMAID'S SECRET,Freida McFadden,The second book in the Housemaid series. The s...,0349132615,9780349132617,2024-11-16,trade-fiction-paperback
9,LIGHTS OUT,Navessa Allen,"As Aly and Josh live out their dark fantasies,...",1638932239,9781638932239,2024-11-16,trade-fiction-paperback


In [11]:
title_lst = []
author_lst = []
summary_lst = []
isbn10_lst = []
isbn13_lst = []

for i in range(0, len(data["results"])):
    title_lst.append(json.dumps(data["results"]["books"][i]["title"], indent=4, sort_keys=True)) # title
    author_lst.append(json.dumps(data["results"]["books"][i]["author"], indent=4, sort_keys=True)) # author
    summary_lst.append(json.dumps(data["results"]["books"][i]["description"], indent=4, sort_keys=True)) # summary
    isbn10_lst.append(json.dumps(data["results"]["books"][i]["primary_isbn10"], indent=4, sort_keys=True)) # isbn10
    isbn13_lst.append(json.dumps(data["results"]["books"][i]["primary_isbn13"], indent=4, sort_keys=True)) # isbn13


In [206]:
book_details_dict = {
    'title': title_lst
    , 'author': author_lst
    , 'summary': summary_lst
    , 'isbn10': isbn10_lst
    , 'isbn13': isbn13_lst
    }

book_details_df = pd.DataFrame(book_details_dict)

book_details_df['title'] = book_details_df['title'].str.strip('"')
book_details_df['author'] = book_details_df['author'].str.strip('"')
book_details_df['summary'] = book_details_df['summary'].str.strip('"')
book_details_df['isbn10'] = book_details_df['isbn10'].str.strip('"')
book_details_df['isbn13'] = book_details_df['isbn13'].str.strip('"')

sales_ending_week = datetime.datetime.strptime(date, "%Y-%m-%d") - timedelta(days=15) # calcuate date of week when sales end
        
book_details_df['sales_ending_week'] = sales_ending_week.strftime("%Y-%m-%d")
book_details_df['NYT_category'] = 'trade-fiction-paperback'

In [207]:
book_details_df
pd.concat([book_details_df, NYT_bestsellers_24], ignore_index=True)

,title,author,summary,isbn10,isbn13,sales_ending_week,NYT_category
0,THE BOYFRIEND,Freida McFadden,A series of recent deaths causes Sydney Shaw t...,1728296226,9781728296227,2024-11-16,trade-fiction-paperback
1,A COURT OF THORNS AND ROSES,Sarah J. Maas,"After killing a wolf in the woods, Feyre is ta...",1635575567,9781635575569,2024-11-16,trade-fiction-paperback
2,THE HOUSEMAID,Freida McFadden,Troubles surface when a woman looking to make ...,1538742578,9781538742570,2024-11-16,trade-fiction-paperback
3,FOURTH WING,Rebecca Yarros,Violet Sorrengail is urged by the commanding g...,1649377371,9781649377371,2024-11-16,trade-fiction-paperback
4,HOW MY NEIGHBOR STOLE CHRISTMAS,Meghan Quinn,A fake relationship and a Christmas contest ma...,1464230153,9781464230158,2024-11-16,trade-fiction-paperback
5,THE FROZEN RIVER,Ariel Lawhon,"In Maine, 1789, a midwife seeks to uncover the...",0593312074,9780593312070,2024-11-16,trade-fiction-paperback
6,THE HANDMAID'S TALE,Margaret Atwood,"In the Republic of Gilead's dystopian future, ...",038549081X,9780385490818,2024-11-16,trade-fiction-paperback
7,HEXED,Emily McIntire,The sixth book in the Never After series. A fo...,1728275865,9781728275864,2024-11-16,trade-fiction-paperback
8,THE HOUSEMAID'S SECRET,Freida McFadden,The second book in the Housemaid series. The s...,0349132615,9780349132617,2024-11-16,trade-fiction-paperback
9,LIGHTS OUT,Navessa Allen,"As Aly and Josh live out their dark fantasies,...",1638932239,9781638932239,2024-11-16,trade-fiction-paperback


In [6]:
#len(NYT_bestsellers_24)
NYT_bestsellers_24

NameError: name 'NYT_bestsellers_24' is not defined